<a href="https://colab.research.google.com/github/mijapa/neuro_linguistic_programming/blob/master/transformers_in_ktrain_T6_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### A Simplied Interface to Text Classification With Hugging Face Transformers in TensorFlow Using [ktrain](https://github.com/amaiya/ktrain)

*ktrain* requires TensorFlow 2.

In [ ]:
!pip3 install -q tensorflow_gpu==2.1.0

     |████████████████████████████████| 421.8MB 31kB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 450kB 35.8MB/s 
     |████████████████████████████████| 3.9MB 52.9MB/s 
ERROR: tensorflow 2.3.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorboard<3,>=2.3.0, but you'll have tensorboard 2.1.1 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorflow-estimator<2.4.0,>=2.3.0, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.


In [ ]:
import tensorflow as tf
print(tf.__version__)

2.1.0


We then need to install *ktrain* library using pip.

In [9]:
!pip3 install -q ktrain

     |████████████████████████████████| 25.3MB 133kB/s 
     |████████████████████████████████| 983kB 52.9MB/s 
     |████████████████████████████████| 266kB 48.4MB/s 
     |████████████████████████████████| 1.3MB 52.8MB/s 
     |████████████████████████████████| 1.1MB 57.2MB/s 
     |████████████████████████████████| 471kB 54.4MB/s 
     |████████████████████████████████| 2.9MB 61.3MB/s 
     |████████████████████████████████| 890kB 60.1MB/s 
ERROR: transformers 3.5.1 has requirement sentencepiece==0.1.91, but you'll have sentencepiece 0.1.94 which is incompatible.


### Load a Dataset Into Arrays

In [3]:
def read_file_as_list(path):
    li = []
    with open(path, 'r') as file:
        for line in file:
            line = line.replace('\n', '')
            li.append(line)
    return li

In [7]:
categories = ['0', '1']


x_train = read_file_as_list('/content/training_set_clean_only_text.txt')
y_train = read_file_as_list('/content/training_set_clean_only_tags.txt')
x_test = read_file_as_list('/content/test_set_clean_only_text.txt')
y_test = read_file_as_list('/content/test_set_clean_only_tags.txt')
print(x_train[0])
print(y_train[0])
print(x_test[0])
print(y_test[0])

Dla mnie faworytem do tytułu będzie Cracovia. Zobaczymy, czy typ się sprawdzi.
0
@anonymized_account Spoko, jak im Duda z Morawieckim zamówią po pięć piw to wszystko będzie ok.
0


## STEP 1:  Preprocess Data and Create a Transformer Model

We will use [DistilBERT](https://arxiv.org/abs/1910.01108).

In [10]:
import ktrain
from ktrain import text
MODEL_NAME = 'distilbert-base-uncased'
t = text.Transformer(MODEL_NAME, maxlen=500, classes=['0', '1'])
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_test(x_test, y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:414: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn('The class_names argument is replacing the classes argument. Please update your code.')



preprocessing train...
language: pl
train sequence lengths:
	mean : 12
	95percentile : 21
	99percentile : 24


/usr/local/lib/python3.6/dist-packages/ktrain/utils.py:589: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


Is Multi-Label? False
preprocessing test...
language: pl
test sequence lengths:
	mean : 12
	95percentile : 21
	99percentile : 23


## STEP 2:  Train the Model

In [11]:
learner.fit_onecycle(5e-5, 4)



begin training using onecycle policy with max lr of 5e-05...
Epoch 1/4
1674/1674 [==============================] - 372s 222ms/step - loss: 0.2899 - accuracy: 0.9143 - val_loss: 0.3473 - val_accuracy: 0.8660
Epoch 2/4
1674/1674 [==============================] - 370s 221ms/step - loss: 0.2757 - accuracy: 0.9150 - val_loss: 0.4204 - val_accuracy: 0.8660
Epoch 3/4
1674/1674 [==============================] - 370s 221ms/step - loss: 0.2888 - accuracy: 0.9152 - val_loss: 0.3743 - val_accuracy: 0.8660
Epoch 4/4
1674/1674 [==============================] - 370s 221ms/step - loss: 0.2338 - accuracy: 0.9153 - val_loss: 0.3407 - val_accuracy: 0.8770


## STEP 3: Evaluate and Inspect the Model

In [12]:
learner.validate(class_names=t.get_classes())

              precision    recall  f1-score   support

           0       0.88      1.00      0.93       866
           1       0.79      0.11      0.20       134

    accuracy                           0.88      1000
   macro avg       0.83      0.55      0.56      1000
weighted avg       0.87      0.88      0.83      1000



array([[862,   4],
       [119,  15]])

Let's examine the validation example about which we were the most wrong.

In [13]:
learner.view_top_losses(n=1, preproc=t)

----------
id:911 | loss:4.61 | true:1 | pred:0)



In [14]:
print(x_test[371])

@anonymized_account Niechce się powtarzać ale znów niewiem czemu var anulowal bramkę


## STEP 4: Making Predictions on New Data in Deployment

In [16]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [32]:
predictor.predict('@anonymized_account Wio endekdu, ścierwa, zdrajcy, szubienica')

'1'

In [33]:
# predicted probability scores for each category
predictor.predict_proba('@anonymized_account Wio endekdu, ścierwa, zdrajcy, szubienica')

array([0.4773687 , 0.52263135], dtype=float32)

In [19]:
predictor.get_classes()

['0', '1']

As expected, `soc.religion.christian` is assigned the highest probability.

Let's invoke the `explain` method to see which words contribute most to the classification.

We will need a forked version of the **eli5** library that supportes TensorFlow Keras, so let's install it first.

In [20]:
!pip3 install -q git+https://github.com/amaiya/eli5@tfkeras_0_10_1


In [34]:
predictor.explain('@anonymized_account Wio endekdu, ścierwa, zdrajcy, szubienica')

The words in the darkest shade of green contribute most to the classification and agree with what you would expect for this example.

We can save and reload our predictor for later deployment.

In [35]:
predictor.save('/tmp/my_distilbert_predictor')

In [ ]:
reloaded_predictor = ktrain.load_predictor('/tmp/my_distilbert_predictor')

In [ ]:
reloaded_predictor.predict('My computer monitor is really blurry.')

'comp.graphics'